In [1]:
! pip install pysqlite3 


Defaulting to user installation because normal site-packages is not writeable
  Using cached pysqlite3-0.5.2.tar.gz (40 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.2-cp312-cp312-linux_x86_64.whl size=168069 sha256=e64bd34d91b91617e4aa91ef31ff133c3c5f873b8d89537cc7b31886fc5bd08b
  Stored in directory: /home/anvil/.cache/pip/wheels/8e/74/94/b439e432c5dac1fa80bd4c6fc443aaaa7370606785abbf1590
Successfully built pysqlite3


In [2]:
import sqlite3
import json

# Path to the Brave browser's history database
history_db = './data/History'

# Connect to the SQLite database
conn = sqlite3.connect(history_db)
c = conn.cursor()

# Query to fetch browsing history
query = "SELECT url, title, visit_count, last_visit_time FROM urls ORDER BY last_visit_time DESC"
c.execute(query)
results = c.fetchall()

# Export to JSON
with open('./data/brave_history.json', 'w', encoding='utf-8') as f:
    history = [{'url': row[0], 'title': row[1], 'visit_count': row[2], 'last_visit_time': row[3]} for row in results]
    json.dump(history, f, ensure_ascii=False, indent=4)

In [3]:
import json
import requests
from urllib.parse import parse_qs, urlparse

# Load the JSON file
with open('./data/brave_history.json', 'r', encoding='utf-8') as f:
    history = json.load(f)

# Function to fetch video title and transcript URL
def get_video_info(video_url):
    video_id = parse_qs(urlparse(video_url).query)['v'][0]
    api_url = f"https://www.youtube.com/get_video_info?video_id={video_id}"
    response = requests.get(api_url)
    data = parse_qs(response.text)

    title = data.get('title', ['No video title'])[0]
    transcript_url = f"https://www.youtube.com/api/timedtext?v={video_id}&lang=en"

    return title, transcript_url

# Iterate through the history and process YouTube URLs
youtube_history = []
for entry in history:
    url = entry['url']
    if 'youtube.com/watch' in url:
        try:
            title, transcript_url = get_video_info(url)
            entry['title'] = title
            entry['transcript_url'] = transcript_url
            youtube_history.append(entry)
        except Exception as e:
            print(f"Error processing URL: {url}")
            print(e)

# Save the modified entries to a new file
with open('./data/youtube_history.json', 'w', encoding='utf-8') as f:
    json.dump(youtube_history, f, ensure_ascii=False, indent=4)

In [6]:
video_id = "LKiBlGDfRU8"
url = f"http://gdata.youtube.com/feeds/api/videos/{video_id}?alt=json&v=2"
print(url)

http://gdata.youtube.com/feeds/api/videos/LKiBlGDfRU8?alt=json&v=2


In [8]:
import dotenv
import os

dotenv.load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')

In [17]:
import requests

url = "https://www.googleapis.com/youtube/v3/videos?part=snippet&id={video_id}&key={google_api_key}"
r = requests.get(url.format(video_id=video_id, google_api_key=google_api_key))
json = r.json()
#print(json)
snippet = json['items'][0]['snippet']
print(snippet)
print(f"Title: {snippet['title']}")

{'publishedAt': '2024-04-05T15:00:30Z', 'channelId': 'UC1yNl2E66ZzKApQdRuTQ4tw', 'title': "My dream died, and now I'm here", 'description': 'This is my contribution to bring the "you" back into YouTube #uinutube\n\nYou can support me on Patreon ➜ https://www.patreon.com/Sabine', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/LKiBlGDfRU8/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/LKiBlGDfRU8/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/LKiBlGDfRU8/hqdefault.jpg', 'width': 480, 'height': 360}, 'standard': {'url': 'https://i.ytimg.com/vi/LKiBlGDfRU8/sddefault.jpg', 'width': 640, 'height': 480}, 'maxres': {'url': 'https://i.ytimg.com/vi/LKiBlGDfRU8/maxresdefault.jpg', 'width': 1280, 'height': 720}}, 'channelTitle': 'Sabine Hossenfelder', 'tags': ['science without the gobbledygoook', 'hossenfelder', 'academia', 'academia is broken', 'academia problems', 'academia crisis', 'science', 'science proble